In [24]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import truncnorm
import time
import csv


def get_s_cond_t_params(player_1_sigma, player_2_sigma, player_1_mean, player_2_mean, t_i):
    beta = 3  # från uppgiften

    # Covariance calculations
    covariance_t_given_s = beta
    covariance_ss = np.array([[player_1_sigma, 0], [0, player_2_sigma]])
    A = np.array([1, -1]).reshape([1, 2])
    inv_covariance_ss = np.linalg.inv(covariance_ss)
    ACA = np.matmul(A.T, A) * (1 / covariance_t_given_s)
    covariance_s_cond_t = np.linalg.inv(inv_covariance_ss + ACA)

    # Mean calculations
    player_means = np.array([player_1_mean, player_2_mean]).reshape([2, 1])
    a = covariance_s_cond_t
    b = np.matmul(inv_covariance_ss, player_means)
    c = A.T * (1 / covariance_t_given_s) * t_i
    mean_s_cond_t = np.matmul(a, b + c)

    return mean_s_cond_t, covariance_s_cond_t


def P_s_cond_t(t_i, player_1_mean, player_1_sigma, player_2_mean, player_2_sigma):
    # player_1_mean = 25; player_2_mean = 25; player_1_sigma = 8.3**2; player_2_sigma = 8.3**2

    mean_s_cond_t, cov_s_cond_t = get_s_cond_t_params(player_1_mean=player_1_mean, player_1_sigma=player_1_sigma,
                                                      player_2_mean=player_2_mean, player_2_sigma=player_2_sigma,
                                                      t_i=t_i)

    return np.random.multivariate_normal(mean=mean_s_cond_t.reshape(2), cov=cov_s_cond_t,
                                         check_valid='warn', tol=1e-8)


def P_t_cond_s(s_i, t_game):
    s_diff = s_i[0] - s_i[1]
    beta = 3
    t_sigma = beta

    if t_game > 0:  # case for when y=1
        a, b = (0 - s_diff) / t_sigma, np.inf
        t = truncnorm.rvs(a, b) * t_sigma + s_diff
        return t
    elif t_game < 0:  # case for when y=-1
        a, b = -np.inf, (0 - s_diff) / t_sigma
        t = truncnorm.rvs(a, b) * t_sigma + s_diff
        return t
    else:
        print("ERROR, TIES PRESENTLY NOT ALLOWED")


        
def gibbs_sampler(L, player_1_stats, player_2_stats, t_game):
    player_1_mean, player_1_sigma = player_1_stats
    player_2_mean, player_2_sigma = player_2_stats
    
    s_i = [player_1_mean, player_2_mean]

    t_obs = np.zeros(L)
    s_obs = np.zeros([L, 2])

    for i in range(L):
        t_i_plus_1 = P_t_cond_s(s_i, t_game=t_game)
        s_i_plus_1 = P_s_cond_t(t_i_plus_1, player_1_mean, player_1_sigma, player_2_mean, player_2_sigma)

        t_obs[i] = t_i_plus_1
        s_obs[i, :] = s_i_plus_1

        s_i = s_i_plus_1
        # plt.scatter(s_obs[:, 0], s_obs[:, 1])
        # plt.pause(0.1)
    # plt.show()

    # plt.plot(s_obs[:, 0]); plt.plot(s_obs[:, 1]); plt.show()
    return s_obs, t_obs



def player_stats_estimate_from_obs(s_obs):
    player_1_stats_estimate = [np.mean(s_obs[:, 0]), np.var(s_obs[:, 0])] # [mean, variance] of samples
    player_2_stats_estimate = [np.mean(s_obs[:, 1]), np.var(s_obs[:, 1])] # [mean, variance] of samples
    
    return player_1_stats_estimate, player_2_stats_estimate


    

def make_stats_dictionary(filename, stats_dictionary, printable):
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        mean = 100; variance = 15 ** 2  # TrueSkill prior parameters before any games
        gamelist = []
        for row in reader:
            # Create dictionary (=map) with keyword 'team' and value [mean, variance]
            stats_dictionary[row['team1']] = [mean, variance]
            # Add teams and result to list
            gamelist.append([row['team1'], row['team2'], int(row['score1']) - int(row['score2'])])

            if printable == 1:  # Print the whole list
                print(f"{row['team1']} vs {row['team2']}: {row['score1']}-{row['score2']}")  # Access by column header
                      
    #stats_dictionary = random.shuffle(stats_dictionary, random)

    return stats_dictionary, gamelist



# Ranking by mean (should perhaps be improved to mean - 3 * sigma)
def ranking(stats_dictionary):
    sorted_teams = sorted(stats_dictionary.items(), key=lambda x: x[1], reverse=True)
    print("\nList of teams ranked by mean skill in descending order:\n")
    for i in sorted_teams:
        print(i[0], i[1])

def played_before(team1, team2, result_list,tmax):
    for t in range(tmax):
        if result_list[t][0] == team2 and result_list[t][1] == team1:
            return -np.sign(result_list[t][2])
    return -999
                      
def get_form(team, form_dictionary):
    if len(form_dictionary[team]) == 0:
        return 0
    elif len(form_dictionary[team]) < 5:
        return sum(form_dictionary[team]) / len(form_dictionary[team])
    else:
        return sum(form_dictionary[team][5:]) / 5
                      
                      
def predict_winner(team1_string, team2_string, stats_dictionary, form_dictionary, result_list, tmax):
    alfa = 0.05  #team 1 home advantage 5 % boost
                      
    form1 = get_form(team1_string,form_dictionary)
    form2 = get_form(team2_string,form_dictionary)
    beta = 0.1
    
    pb = played_before(team1_string, team2_string, result_list, tmax)
    if  pb == -999:
        gamma = 0
    else: 
        gamma = 0.05
                      
    s1 = stats_dictionary[team1_string][0]
    s2 = stats_dictionary[team2_string][0]

    p1 = s1 + alfa*abs(s1) + beta*form1*abs(s1) + gamma*pb*abs(s1)  #prediction score team 1
    p2 = s2 + beta*form2*abs(s2) + gamma*(-pb)*abs(s2)   #prediction score team 2
                      
    diff = p1 - p2
    print('\t diff: ', diff)
    
    if diff > 0:
        return 1
    elif diff < 0:
        return -1
    else:
        return 1

In [25]:
# Make dictionary of team stats (mean and variance) & list of all games with result
stats_dictionary, result_list = make_stats_dictionary('SerieA.csv', stats_dictionary={}, printable=0)
stats_dictionary

{'Chievo': [100, 225],
 'Lazio': [100, 225],
 'Torino': [100, 225],
 'Sassuolo': [100, 225],
 'Parma': [100, 225],
 'Empoli': [100, 225],
 'Bologna': [100, 225],
 'Atalanta': [100, 225],
 'Juventus': [100, 225],
 'Napoli': [100, 225],
 'Spal': [100, 225],
 'Udinese': [100, 225],
 'Inter': [100, 225],
 'Genoa': [100, 225],
 'Frosinone': [100, 225],
 'Fiorentina': [100, 225],
 'Cagliari': [100, 225],
 'Roma': [100, 225],
 'Milan': [100, 225],
 'Sampdoria': [100, 225]}

In [26]:
stats_dictionary, result_list = make_stats_dictionary('SerieA.csv', stats_dictionary={}, printable=0)
form_dictionary = dict() # keep track of the teams 5-game-form.
for keys in stats_dictionary:
    form_dictionary[keys] = []
#form_dictionary

In [27]:
correct_predictions = 0
nr_of_draws = 0

for i in range(len(result_list)):
    # result_list holds [team1, team2, result = score1 - score2]
    print(f"\nResult game {i}:   {result_list[i]}")
    team1 = result_list[i][0]
    team2 = result_list[i][1]
    result = result_list[i][2]
    
    # stats_dictionary with keyword 'teamname' and value [mean, variance]
    print(f"Stats before game: {stats_dictionary[team1]}, {stats_dictionary[team2]}")

    if result == 0:  # ignore tied games for now
        print("Game ignored due to tie")
        print(f"Stats after game:  {stats_dictionary[team1]}, {stats_dictionary[team2]}")
        nr_of_draws += 1
        print("Draws: ", nr_of_draws)
        
        form_dictionary[team1].append(0)
        form_dictionary[team2].append(0)
    else:
    
        # predict result and save prediction
        pred_result = predict_winner(team1, team2, stats_dictionary, form_dictionary, result_list, tmax=i)
        
        if np.sign(result) == np.sign(pred_result):
                correct_predictions += 1
                print("Correct pred: ", correct_predictions)
        
        print('pred result: ', pred_result)
        print('true result: ', result)
                
        form_dictionary[team1].append(np.sign(result))
        form_dictionary[team2].append(-np.sign(result))
        #print('form dic 1:', form_dictionary[team1])
        #print('form dic 2:', form_dictionary[team2])
            
        s_obs, t_obs =  gibbs_sampler(L=5000,
                                      player_1_stats=stats_dictionary[team1], 
                                      player_2_stats=stats_dictionary[team2],
                                      t_game=result)
        
        burn_in_value = 3000
        s_obs = s_obs[burn_in_value:]
        player_1_stats_posterior, player_2_stats_posterior = player_stats_estimate_from_obs(s_obs)
        
        
        # Update team stats so posterior makes new prior
        stats_dictionary[team1] = player_1_stats_posterior
        stats_dictionary[team2] = player_2_stats_posterior


        print(f"Stats AFTER game: {stats_dictionary[team1]}, {stats_dictionary[team2]}")

print()
print("correct_predictions ", correct_predictions)
print("reslist: ", len(result_list))
print("draws: ", nr_of_draws)
print('performance: ', correct_predictions / (len(result_list)-nr_of_draws))



Result game 0:   ['Chievo', 'Juventus', -1]
Stats before game: [100, 225], [100, 225]
	 diff:  5.0
pred result:  1
true result:  -1
Stats AFTER game: [86.35957602033834, 188.9996297974151], [113.74145893682837, 195.3973154688803]

Result game 1:   ['Lazio', 'Napoli', -1]
Stats before game: [100, 225], [100, 225]
	 diff:  5.0
pred result:  1
true result:  -1
Stats AFTER game: [84.8134686928642, 229.94327498286776], [115.46352591270637, 235.36536305764602]

Result game 2:   ['Torino', 'Roma', -1]
Stats before game: [100, 225], [100, 225]
	 diff:  5.0
pred result:  1
true result:  -1
Stats AFTER game: [87.37022450772501, 171.554044379432], [112.02614115771301, 180.94621481293427]

Result game 3:   ['Sassuolo', 'Inter', 1]
Stats before game: [100, 225], [100, 225]
	 diff:  5.0
Correct pred:  1
pred result:  1
true result:  1
Stats AFTER game: [109.92134313716294, 180.6197007206105], [89.51117627300727, 189.48698310182425]

Result game 4:   ['Parma', 'Udinese', 0]
Stats before game: [100, 

Stats AFTER game: [113.34223615567625, 132.3209715034124], [99.71978279712678, 131.18389927429405]

Result game 30:   ['Frosinone', 'Sampdoria', -5]
Stats before game: [72.1025849095973, 173.69752133399493], [106.49032330145758, 129.64162898976844]
	 diff:  -35.58944814035358
Correct pred:  11
pred result:  -1
true result:  -5
Stats AFTER game: [71.20526049866848, 198.8460096878546], [107.43106286762186, 148.00892678207887]

Result game 31:   ['Roma', 'Chievo', 0]
Stats before game: [93.9612283600466, 178.91221318049605], [80.01859703112734, 197.90756859670245]
Game ignored due to tie
Stats after game:  [93.9612283600466, 178.91221318049605], [80.01859703112734, 197.90756859670245]
Draws:  7

Result game 32:   ['Udinese', 'Torino', 0]
Stats before game: [99.47878291235905, 146.547911038906], [111.00984285948611, 98.39352105057318]
Game ignored due to tie
Stats after game:  [99.47878291235905, 146.547911038906], [111.00984285948611, 98.39352105057318]
Draws:  8

Result game 33:   ['Juve

Stats AFTER game: [92.99556895476394, 76.10941859289457], [114.5366842243959, 89.46667781048866]

Result game 58:   ['Empoli', 'Milan', 0]
Stats before game: [88.60637437451979, 114.1251227435357], [120.72244594843191, 318.0045526571233]
Game ignored due to tie
Stats after game:  [88.60637437451979, 114.1251227435357], [120.72244594843191, 318.0045526571233]
Draws:  14

Result game 59:   ['Roma', 'Lazio', 2]
Stats before game: [81.5430217887331, 119.17436017365999], [120.44805343368428, 110.0673976571435]
	 diff:  -35.60598118841355
pred result:  -1
true result:  2
Stats AFTER game: [106.85615258471577, 81.34930959004625], [97.28576181916831, 81.45233035055077]

Result game 60:   ['Juventus', 'Napoli', 2]
Stats before game: [133.13980766844813, 233.1515640605789], [127.44360759596425, 130.20357062424338]
	 diff:  12.467114457355933
Correct pred:  26
pred result:  1
true result:  2
Stats AFTER game: [143.17329578519204, 187.9138061796656], [122.13265475718576, 116.90849643792188]

Resul

Stats AFTER game: [51.64486588156141, 177.36431322690473], [74.15138955780515, 142.19889684069804]

Result game 85:   ['Bologna', 'Torino', 0]
Stats before game: [91.91486294047793, 109.3642367350878], [106.18564086245117, 122.86081055695217]
Game ignored due to tie
Stats after game:  [91.91486294047793, 109.3642367350878], [106.18564086245117, 122.86081055695217]
Draws:  17

Result game 86:   ['Fiorentina', 'Cagliari', 0]
Stats before game: [92.1211072399035, 89.7482218877496], [104.99175559454658, 84.10971900296427]
Game ignored due to tie
Stats after game:  [92.1211072399035, 89.7482218877496], [104.99175559454658, 84.10971900296427]
Draws:  18

Result game 87:   ['Inter', 'Milan', 1]
Stats before game: [119.94202190904164, 125.13727552454615], [153.4355615725128, 380.0755960700301]
	 diff:  -26.437339716377068
pred result:  -1
true result:  1
Stats AFTER game: [131.03913603818495, 96.65253508042431], [119.1369250842793, 146.1544093364486]

Result game 88:   ['Sampdoria', 'Sassuolo'

Stats AFTER game: [95.36695415691742, 158.88024603971917], [120.77625142242928, 103.20539233134042]

Result game 114:   ['Atalanta', 'Inter', 3]
Stats before game: [104.96597803957133, 67.61342613019251], [137.1257211870205, 210.32690421291602]
	 diff:  -41.26721122712162
pred result:  -1
true result:  3
Stats AFTER game: [116.2254370940425, 59.992023928168514], [102.38833218813174, 116.05838415404119]

Result game 115:   ['Roma', 'Sampdoria', 3]
Stats before game: [92.03623188000778, 59.46448038000308], [94.66036897860702, 79.95663095643887]
	 diff:  5.659123770601454
Correct pred:  55
pred result:  1
true result:  3
Stats AFTER game: [98.74006328552464, 49.188275086870604], [85.77027855367685, 67.96540383156447]

Result game 116:   ['Empoli', 'Udinese', 1]
Stats before game: [88.99552366913842, 190.41382433204538], [84.30718458896122, 175.85531723614946]
	 diff:  10.449191828999162
Correct pred:  56
pred result:  1
true result:  1
Stats AFTER game: [104.55216839949856, 195.1111708058

...

Stats AFTER game: [109.77726066704659, 24.733815880291825], [104.5879101493858, 20.60904250042905]

Result game 253:   ['Torino', 'Chievo', 3]
Stats before game: [116.7719110966996, 33.16666143653463], [60.24549042727578, 412.4616022824564]
	 diff:  94.94217549558266
Correct pred:  117
pred result:  1
true result:  3
Stats AFTER game: [116.6558716339859, 34.40299106074647], [61.036743467349034, 433.30372233246743]

Result game 254:   ['Udinese', 'Bologna', 1]
Stats before game: [94.33835947045931, 39.907426868378245], [97.96664320404349, 51.28662673396162]
	 diff:  -7.946090496412879
pred result:  -1
true result:  1
Stats AFTER game: [100.0501866235886, 30.890004325976378], [90.94114098005237, 36.44898090126556]

Result game 255:   ['Spal', 'Sampdoria', -1]
Stats before game: [97.54232254570375, 27.57133897979228], [101.92510688545482, 22.423614521259225]
	 diff:  -27.124313429045515
Correct pred:  118
pred result:  -1
true result:  -1
Stats AFTER game: [94.40137513040482, 26.177448286

Stats AFTER game: [55.95977346291188, 793.2728391714027], [101.0487773177213, 11.359767905470365]

Result game 280:   ['Udinese', 'Genoa', 2]
Stats before game: [97.22675774369057, 42.23876206765909], [96.10650858711924, 17.88306466059129]
	 diff:  -3.830708663138907
pred result:  -1
true result:  2
Stats AFTER game: [104.1094346252143, 47.25092090972694], [93.34419817342814, 18.531473408684715]

Result game 281:   ['Juventus', 'Empoli', 1]
Stats before game: [83.22555945031775, 127.14417607275203], [103.4747671237688, 42.950671083286885]
	 diff:  37.69475542721118
Correct pred:  130
pred result:  1
true result:  1
Stats AFTER game: [105.38510713977932, 80.93683233856108], [95.8140436209936, 37.28262222125399]

Result game 282:   ['Sampdoria', 'Milan', 1]
Stats before game: [104.31028591692808, 24.307093494018417], [94.71552968509062, 57.38939178804812]
	 diff:  -8.017404159056255
pred result:  -1
true result:  1
Stats AFTER game: [105.71161188542206, 25.90844333547776], [91.1215276821

Stats AFTER game: [109.03373192975872, 18.55596664623781], [40.99963619021175, 1879.0196166412989]

Result game 309:   ['Spal', 'Juventus', 1]
Stats before game: [101.70514890087819, 8.938082831313782], [113.08385028551554, 82.14876670211379]
	 diff:  -80.80093484297018
pred result:  -1
true result:  1
Stats AFTER game: [103.39235965509081, 8.058461438530706], [96.82048975262053, 32.64880189096979]

Result game 310:   ['Roma', 'Udinese', 1]
Stats before game: [100.67526119954432, 11.507276469369398], [108.355494082904, 37.56472028752085]
	 diff:  16.166298152037882
Correct pred:  143
pred result:  1
true result:  1
Stats AFTER game: [103.69709107689833, 10.009899487685775], [98.25829342765819, 21.421562605408727]

Result game 311:   ['Milan', 'Lazio', 1]
Stats before game: [90.86481295014478, 61.80019649523202], [102.54785806254587, 18.783984480638182]
	 diff:  -2.622559296877256
pred result:  -1
true result:  1
Stats AFTER game: [105.39766029469449, 40.91849863497763], [98.24587223358

Stats AFTER game: [94.65799321634587, 16.927760960190593], [102.5303706406252, 13.518623633949074]

Result game 337:   ['Torino', 'Milan', 2]
Stats before game: [108.45303931111785, 23.363616078581742], [105.39766029469449, 40.91849863497763]
	 diff:  8.966891624606973
Correct pred:  157
pred result:  1
true result:  2
Stats AFTER game: [111.04831265801633, 21.6482231393788], [101.04120019638151, 36.844979712695924]

Result game 338:   ['Atalanta', 'Udinese', 2]
Stats before game: [123.73007800144862, 80.05597159451786], [94.39048053226, 17.77143684820493]
	 diff:  83.21922827198507
Correct pred:  158
pred result:  1
true result:  2
Stats AFTER game: [123.95077332247371, 115.12229508580377], [94.23549562179615, 19.084608887771665]

Result game 339:   ['Fiorentina', 'Sassuolo', -1]
Stats before game: [89.47689415515683, 19.48181483659004], [92.14639131764831, 60.33227265688364]
	 diff:  5.650373027721784
pred result:  1
true result:  -1
Stats AFTER game: [87.8182727556364, 19.4035528600

Stats AFTER game: [94.87001979665901, 10.370534419950916], [84.4657767128272, 27.066848147705738]

Result game 365:   ['Empoli', 'Torino', 3]
Stats before game: [101.3912933703057, 29.530373333114373], [112.06833834035409, 23.73579045039029]
	 diff:  -54.916736494385844
pred result:  -1
true result:  3
Stats AFTER game: [110.13991427558035, 20.49689223602662], [104.98004055849695, 17.63021974276349]

Result game 366:   ['Milan', 'Frosinone', 2]
Stats before game: [109.74086623964136, 32.87714806271328], [90.73732808959187, 15.180235981100028]
	 diff:  67.8356426884609
Correct pred:  170
pred result:  1
true result:  2
Stats AFTER game: [110.06661011083811, 46.76882292592081], [90.57980931923798, 17.586071434099605]

Result game 367:   ['Napoli', 'Inter', 3]
Stats before game: [113.99372579746488, 34.662004157003075], [106.39762921564561, 15.43611001434594]
	 diff:  10.786979916885116
Correct pred:  171
pred result:  1
true result:  3
Stats AFTER game: [115.99740633254324, 31.4405303769

In [28]:
ranking(stats_dictionary)


List of teams ranked by mean skill in descending order:

Atalanta [127.56853047597778, 201.02308712070536]
Milan [112.80704620917814, 54.77590389080689]
Bologna [111.97076165380982, 16.678163027030774]
Inter [109.61006737804834, 12.461281708383751]
Torino [108.42277139527204, 13.60282109383998]
Roma [107.64871635518467, 10.878394621748834]
Napoli [106.57933930290602, 20.493593494633064]
Udinese [106.51960336424715, 12.885496730575197]
Empoli [104.49997068821057, 15.795204709315975]
Lazio [102.76520277630914, 10.901174140957938]
Spal [100.84120978144314, 6.459196362286172]
Cagliari [99.16934765214086, 6.6715106149033]
Sampdoria [96.45508350797213, 13.652000988578244]
Sassuolo [96.35836778038042, 43.530834604381596]
Parma [95.03907021587487, 13.23708049331079]
Juventus [90.69365722123518, 20.58686139638447]
Frosinone [90.57980931923798, 17.586071434099605]
Genoa [88.59705976082167, 29.257460445696772]
Fiorentina [84.4657767128272, 27.066848147705738]
Chievo [37.022309200637885, 1164.812